In [1]:
!pip install surfboard

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.8/276.8 KB 16.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.2/317.2 KB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 80.8 MB/s eta 0:00:00
  Created wheel for surfboard: filename=surfboard-0.2.0-py3-none-any.whl size=33226 sha256=9c1b5b983a8baa6a50317c19ee2c76d8b861de2dcf579434525ef8ad1c2fdd3e
  Stored i

In [2]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns

from sklearn import preprocessing


import librosa
import librosa.display
import IPython.display


from google.colab import drive

from tqdm import tqdm

from surfboard import feature_extraction


In [3]:
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
path_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Project/path_train.csv')
path_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Project/path_test.csv')
y_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Project/y_train.csv', usecols=['target'])
y_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Project/y_test.csv', usecols=['target'])

In [ ]:
!pip install openl3
import openl3

In [ ]:
y_train.head()

In [ ]:
path_train.head()

In [ ]:
path_train.shape

In [ ]:
config = {
    'log_melspec': {
        'hop_length_seconds': 0.02,
        'n_fft_seconds': 0.08,
        'n_mels': 64,
    },
    'loudness_slidingwindow': {
        'frame_length_seconds': 1.0,
        'hop_length_seconds': 0.25
    }
}

In [ ]:
path_train

In [ ]:
path_train['path'] = '/content/drive/MyDrive/Colab Notebooks/Project/pneumonia_files' + path_train['path'].str.lstrip('.')

In [ ]:
path_test['path'] = '/content/drive/MyDrive/Colab Notebooks/Project/pneumonia_files' + path_test['path'].str.lstrip('.')

In [ ]:
path_test['path']

In [ ]:
X_train = (
    feature_extraction.extract_features_from_paths(
        path_train['path'],
        components_list=[{key: config[key]} for key in config],
        statistics_list=["mean", "std"],
    )
    .replace(-np.inf, np.nan)
    .fillna(method="bfill")
)

In [ ]:
X_train.shape

In [ ]:
X_train

In [ ]:
y_train.shape

In [ ]:
y_test.shape

In [ ]:
y_train

In [ ]:
X_test = (
    feature_extraction.extract_features_from_paths(
        path_test['path'],
        components_list=[{key: config[key]} for key in config],
        statistics_list=["mean", "std"],
    )
    .replace(-np.inf, np.nan)
    .fillna(method="bfill"))

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score


for estim in range(50, 200, 5):
    clf = AdaBoostClassifier(n_estimators=estim, random_state=99, algorithm='SAMME.R')
    %time clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    recall = precision_recall_fscore_support(y_test, y_pred, average='binary')[0]
    precision = precision_recall_fscore_support(y_test, y_pred, average='binary')[1]
    accuracy = accuracy_score(y_test, y_pred)
    print(f"\nFor number of estimators {estim} Precision {precision}, Recall {recall} Accuracy {accuracy}\n")

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
y_pred.shape

In [ ]:
clf = AdaBoostClassifier(n_estimators=95, random_state=99, algorithm='SAMME.R')
clf.fit(X_train, y_train)

clf.score(X_test, y_test)

In [ ]:
y_score = clf.predict_proba(X_test)

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
from sklearn.metrics import r2_score

In [ ]:
fpr, tpr, threshold = roc_curve(y_test, y_score[:, 1])
roc_auc = auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show();

print(f"ROC AUC is {roc_auc}")

In [ ]:
precision, recall, threshold = precision_recall_curve(y_test, y_score[:, 1])

fig, ax = plt.subplots()
ax.plot(recall, precision, color='purple')
ax.set_title('Precision-Recall Curve')
ax.set_ylabel('Precision')
ax.set_xlabel('Recall')
plt.show();

In [ ]:
for value in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    idx = (np.abs(recall - value)).argmin() #index for recall
    result = np.where(precision == precision[idx]) # indexes for precision
    print(f"\nPrecision for recall = {value} is {precision[result]}")
    try:
      print(f"Threshold for recall = {value} is {threshold[result]}")
    except:
      print('none')

In [ ]:
threshold = 0.498
y_pred_final = np.where(np.array(y_score[:, 1]) < threshold, 0, 1)
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred_final)
ConfusionMatrixDisplay(confusion_matrix=cm).plot();

In [ ]:
pd.DataFrame(y_pred_final).to_csv("/content/drive/MyDrive/Colab Notebooks/Project/y_pred_final.csv")
pd.DataFrame(y_pred).to_csv("/content/drive/MyDrive/Colab Notebooks/Project/y_pred.csv")

In [ ]:
fp_indices = np.where((y_pred_final.reshape(-1,1)==1)&(y_test==0))
tp_indices = np.where((y_pred_final.reshape(-1,1)==1)&(y_test==1))

In [ ]:
false_pos = path_test.loc[fp_indices[0], :]

In [ ]:
true_pos = path_test.loc[tp_indices[0], :]

In [ ]:
path_test.loc[fp_indices[0], :].to_csv('false_pos.csv')

In [ ]:
path_test.loc[tp_indices[0], :].to_csv('true_pos.csv')

In [ ]:
false_pos.head()

In [ ]:
features = pd.DataFrame(false_pos[['id', 'model', 'point']].describe().loc['unique'])


fig, ax = plt.subplots()
bars = ax.bar(features.index, features['unique'])
ax.bar_label(bars)
plt.xlabel('Feature', )
plt.ylabel('Number of unique values')
plt.title('Number of uniqie values for categorical features')
plt.xticks(rotation=45, ha='right')
plt.show();

In [ ]:
# define subplot grid
fig = plt.figure(figsize=(20, 6))
sns.histplot(false_pos['id'])
plt.title('ID')
ax.set_xlabel('id')
plt.xticks(rotation=45, ha='right')
plt.show();

In [ ]:

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(20, 6))

plt.title("Models for FP and TP", fontsize=18, y=1)

sns.histplot(false_pos['model'], ax=ax[0])
ax[0].set_title('False Pos')
ax[0].set_xlabel('Models')
ax[0].tick_params('x', rotation=90, left=True)
sns.histplot(true_pos['model'], ax=ax[1])
ax[1].set_title('True Pos')
ax[1].set_xlabel('Models')
ax[1].tick_params(rotation=90)
plt.show();

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(20, 8))

plt.title("Position for FP and TP", fontsize=18, y=1)

sns.histplot(false_pos['point'], ax=ax[0])
ax[0].set_title('False Pos')
ax[0].set_xlabel('Point')
ax[0].tick_params(rotation=90)
sns.histplot(true_pos['point'], ax=ax[1])
ax[1].set_title('True Pos')
ax[1].set_xlabel('Point')
ax[1].tick_params(rotation=90)
plt.show();

In [ ]:
u_true, c_true = np.unique(true_pos['id'], return_counts=True)
u_false, c_false = np.unique(false_pos['id'], return_counts=True)

# Check which unique elements are common in both arrays
print('Ids intersect in', np.intersect1d(u_true,u_false))

In [ ]:
u_true_model, c_true_model = np.unique(true_pos['model'], return_counts=True)
u_false_model, c_false_model = np.unique(false_pos['model'], return_counts=True)

# Check which unique elements are common in both arrays
print('Models intersect in', np.intersect1d(u_true_model,u_false_model), 'counts are: ', c_true_model[np.intersect1d(u_true_model,u_false_model, return_indices=True)[1]])

In [ ]:
df1_model = pd.DataFrame(np.intersect1d(u_true_model,u_false_model), columns=['model'])
df2_model = pd.DataFrame(c_true_model[np.intersect1d(u_true_model,u_false_model, return_indices=True)[1]], columns=['count'])

In [ ]:
df_model = pd.concat([df1_model, df2_model], axis=1)

In [ ]:
print('these models are showing FP and TP')
df_model

In [ ]:
diff_true_model = np.setdiff1d(u_true_model,u_false_model)
print('These models are only showing True' ,diff_true_model)

diff_false_model = np.setdiff1d(u_false_model, u_true_model)
print('These models are only showing False' ,diff_false_model)

In [ ]:
false_pos[false_pos['model'].isin(diff_false_model)]

In [ ]:
true_pos[true_pos['model'].isin(diff_true_model)]

In [ ]:
fig = plt.figure(figsize=(20, 8))

plt.title("Model for FP and TP", fontsize=18, y=1)

sns.histplot(false_pos['model'], alpha=0.5, color='r', label='false')
sns.histplot(true_pos['model'], alpha=0.5, color='g', label='true')

plt.legend()
plt.tick_params(rotation=90)
plt.show();

In [ ]:
u_true_point, c_true_point = np.unique(true_pos['point'], return_counts=True)
u_false_point, c_false_point = np.unique(false_pos['point'], return_counts=True)

# Check which unique elements are common in both arrays
print('Models intersect in', np.intersect1d(u_true_point,u_false_point), 'counts are: ', c_true_point[np.intersect1d(u_true_point,u_false_point, return_indices=True)[1]])

In [ ]:
df1_point = pd.DataFrame(np.intersect1d(u_true_point,u_false_point), columns=['point'])
df2_point = pd.DataFrame(c_true_point[np.intersect1d(u_true_point,u_false_point, return_indices=True)[1]], columns=['count'])

In [ ]:
df_point = pd.concat([df1_point, df2_point], axis=1)

In [ ]:
print('these points are showing FP and TP')
df_point

In [ ]:
diff_true_point = np.setdiff1d(u_true_point,u_false_point)
print('These models are only showing True' ,diff_true_point)

diff_false_point = np.setdiff1d(u_false_point, u_true_point)
print('These models are only showing False' ,diff_false_point)

In [ ]:
fig = plt.figure(figsize=(20, 8))

plt.title("Point for FP and TP", fontsize=18, y=1)

sns.histplot(false_pos['point'], alpha=0.5, color='r', label='false')
sns.histplot(true_pos['point'], alpha=0.5, color='g', label='true')

plt.legend()
plt.tick_params(rotation=90)
plt.show();

In [ ]:
X_train.columns

In [ ]:
from sklearn.svm import SVC
for kernel in  ['linear', 'poly', 'rbf', 'sigmoid']:
  classifier_SVM = SVC(kernel = kernel, probability=True)
  classifier_SVM.fit(X_train, y_train)
  y_pred = classifier_SVM.predict(X_test)
  y_score = classifier_SVM.predict_proba(X_test)
  print(f"Accuracy for kernel {kernel} is  {accuracy_score(y_test, y_pred)}")


In [ ]:
classifier_SVM = SVC(kernel = 'linear', probability=True)
classifier_SVM.fit(X_train, y_train)
y_pred = classifier_SVM.predict(X_test)
y_score = classifier_SVM.predict_proba(X_test)

In [ ]:
fpr, tpr, threshold = roc_curve(y_test, y_score[:, 1])
roc_auc = auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show();

print(f"ROC AUC is {roc_auc}")

In [ ]:
threshold = 0.5
y_pred_final = np.where(np.array(y_score[:, 1]) < threshold, 0, 1)
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred_final)
display(cm)
ConfusionMatrixDisplay(confusion_matrix=cm).plot();